In [9]:
import pandas as pd

prices = pd.read_excel('Price.xlsx')
prices = prices.loc[prices['PriceArea'] == 'DK1']
prices = prices['PriceDKK'].values

In [10]:
import numpy as np


ACTIONS = ["Charge", "Discharge", "Do nothing"]
PRICES_STATE = ["Low", "Medium", "High"]
SOC_STATE = ["LOW", "MEDIUM", "HIGH"]

P_ssa = np.zeros((len(PRICES_STATE), len(SOC_STATE), len(ACTIONS)))

array([[[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]],

       [[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]]])